### Required Assignment 5.1: Will the Customer Accept the Coupon?

**Context**

Imagine driving through town and a coupon is delivered to your cell phone for a restaurant near where you are driving. Would you accept that coupon and take a short detour to the restaurant? Would you accept the coupon but use it on a subsequent trip? Would you ignore the coupon entirely? What if the coupon was for a bar instead of a restaurant? What about a coffee house? Would you accept a bar coupon with a minor passenger in the car? What about if it was just you and your partner in the car? Would weather impact the rate of acceptance? What about the time of day?

Obviously, proximity to the business is a factor on whether the coupon is delivered to the driver or not, but what are the factors that determine whether a driver accepts the coupon once it is delivered to them? How would you determine whether a driver is likely to accept a coupon?

**Overview**

The goal of this project is to use what you know about visualizations and probability distributions to distinguish between customers who accepted a driving coupon versus those that did not.

**Data**

This data comes to us from the UCI Machine Learning repository and was collected via a survey on Amazon Mechanical Turk. The survey describes different driving scenarios including the destination, current time, weather, passenger, etc., and then ask the person whether he will accept the coupon if he is the driver. Answers that the user will drive there ‘right away’ or ‘later before the coupon expires’ are labeled as ‘Y = 1’ and answers ‘no, I do not want the coupon’ are labeled as ‘Y = 0’.  There are five different types of coupons -- less expensive restaurants (under \$20), coffee houses, carry out & take away, bar, and more expensive restaurants (\$20 - $50).

**Deliverables**

Your final product should be a brief report that highlights the differences between customers who did and did not accept the coupons.  To explore the data you will utilize your knowledge of plotting, statistical summaries, and visualization using Python. You will publish your findings in a public facing github repository as your first portfolio piece.





### Data Description
Keep in mind that these values mentioned below are average values.

The attributes of this data set include:
1. User attributes
    -  Gender: male, female
    -  Age: below 21, 21 to 25, 26 to 30, etc.
    -  Marital Status: single, married partner, unmarried partner, or widowed
    -  Number of children: 0, 1, or more than 1
    -  Education: high school, bachelors degree, associates degree, or graduate degree
    -  Occupation: architecture & engineering, business & financial, etc.
    -  Annual income: less than \\$12500, \\$12500 - \\$24999, \\$25000 - \\$37499, etc.
    -  Number of times that he/she goes to a bar: 0, less than 1, 1 to 3, 4 to 8 or greater than 8
    -  Number of times that he/she buys takeaway food: 0, less than 1, 1 to 3, 4 to 8 or greater
    than 8
    -  Number of times that he/she goes to a coffee house: 0, less than 1, 1 to 3, 4 to 8 or
    greater than 8
    -  Number of times that he/she eats at a restaurant with average expense less than \\$20 per
    person: 0, less than 1, 1 to 3, 4 to 8 or greater than 8
    -  Number of times that he/she goes to a bar: 0, less than 1, 1 to 3, 4 to 8 or greater than 8
    

2. Contextual attributes
    - Driving destination: home, work, or no urgent destination
    - Location of user, coupon and destination: we provide a map to show the geographical
    location of the user, destination, and the venue, and we mark the distance between each
    two places with time of driving. The user can see whether the venue is in the same
    direction as the destination.
    - Weather: sunny, rainy, or snowy
    - Temperature: 30F, 55F, or 80F
    - Time: 10AM, 2PM, or 6PM
    - Passenger: alone, partner, kid(s), or friend(s)


3. Coupon attributes
    - time before it expires: 2 hours or one day

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

### Problems

Use the prompts below to get started with your data analysis.  

1. Read in the `coupons.csv` file.




In [6]:
data = pd.read_csv('data/coupons.csv')
data.sample(2)

,destination,passanger,weather,temperature,time,coupon,expiration,gender,age,maritalStatus,...,CoffeeHouse,CarryAway,RestaurantLessThan20,Restaurant20To50,toCoupon_GEQ5min,toCoupon_GEQ15min,toCoupon_GEQ25min,direction_same,direction_opp,Y
4088,No Urgent Place,Friend(s),Sunny,80,2PM,Restaurant(<20),2h,Male,21,Unmarried partner,...,less1,4~8,1~3,less1,1,1,0,0,1,1
8013,Work,Alone,Sunny,80,7AM,Restaurant(<20),1d,Male,26,Unmarried partner,...,never,4~8,never,1~3,1,0,0,1,0,1


2. Investigate the dataset for missing or problematic data.

In [4]:
# find null or missing values
columns_with_nulls = data.isna().sum()

# find % of missing values
columns_with_nulls = (columns_with_nulls[columns_with_nulls.values > 0]/len(data))
columns_with_nulls


car                     0.991485
Bar                     0.008436
CoffeeHouse             0.017108
CarryAway               0.011905
RestaurantLessThan20    0.010249
Restaurant20To50        0.014901
dtype: float64

In [11]:
print(data['CoffeeHouse'].value_counts())
print(data['CarryAway'].value_counts())

CoffeeHouse
less1    3385
1~3      3225
never    2962
4~8      1784
gt8      1111
Name: count, dtype: int64
CarryAway
1~3      4672
4~8      4258
less1    1856
gt8      1594
never     153
Name: count, dtype: int64


In [13]:
columns_with_nulls.index

Index(['car', 'Bar', 'CoffeeHouse', 'CarryAway', 'RestaurantLessThan20',
       'Restaurant20To50'],
      dtype='object')

3. Decide what to do about your missing data -- drop, replace, other...

In [14]:
# remove car column since car column is missing > 95% of its values

# drop columns where missing values > 95%
clean_data = data.drop(columns = columns_with_nulls[columns_with_nulls > 0.95].index)

# for columns with < 1 % missing values, fill values of the most common value observed in that column in the data set
clean_data['Bar'] = data['Bar'].fillna('never')
clean_data['CoffeeHouse'] = data['CoffeeHouse'].fillna('less1')
clean_data['CarryAway'] = data['CarryAway'].fillna('1~3')
clean_data['RestaurantLessThan20'] = data['RestaurantLessThan20'].fillna('1~3')
clean_data['Restaurant20To50'] = data['Restaurant20To50'].fillna('less1')

# for the columns 'Bar', 'CoffeeHouse', 'CarryAway', 'RestaurantLessThan20','Restaurant20To50' 
#       the value never and less1, appear to denote the same meaning. So we will update the values with never, where less1 is present
clean_data['Bar'] = clean_data['Bar'].map(lambda x: 'never' if x == 'less1' else x)
clean_data['CoffeeHouse'] = clean_data['CoffeeHouse'].map(lambda x: 'never' if x == 'less1' else x)
clean_data['CarryAway'] = clean_data['CarryAway'].map(lambda x: 'never' if x == 'less1' else x)
clean_data['RestaurantLessThan20'] = clean_data['RestaurantLessThan20'].map(lambda x: 'never' if x == 'less1' else x)
clean_data['Restaurant20To50'] = clean_data['Restaurant20To50'].map(lambda x: 'never' if x == 'less1' else x)

clean_data['Y-new'] = clean_data['Y'].map(lambda x: "Coupon Accepted" if x == 1 else "Did Not Accept Coupon")

# drop duplicates
clean_data = clean_data.drop_duplicates()
print(len(clean_data))

clean_data.sample(2)

12596


,destination,passanger,weather,temperature,time,coupon,expiration,gender,age,maritalStatus,...,CarryAway,RestaurantLessThan20,Restaurant20To50,toCoupon_GEQ5min,toCoupon_GEQ15min,toCoupon_GEQ25min,direction_same,direction_opp,Y,Y-new
480,No Urgent Place,Friend(s),Sunny,80,2PM,Restaurant(<20),1d,Male,26,Single,...,4~8,never,never,1,1,0,0,1,1,Coupon Accepted
11514,Work,Alone,Rainy,55,7AM,Carry out & Take away,1d,Female,below21,Single,...,never,4~8,gt8,1,0,0,0,1,0,Did Not Accept Coupon


4. What proportion of the total observations chose to accept the coupon?



5. Use a bar plot to visualize the `coupon` column.

6. Use a histogram to visualize the temperature column.

**Investigating the Bar Coupons**

Now, we will lead you through an exploration of just the bar related coupons.  

1. Create a new `DataFrame` that contains just the bar coupons.


2. What proportion of bar coupons were accepted?


3. Compare the acceptance rate between those who went to a bar 3 or fewer times a month to those who went more.


4. Compare the acceptance rate between drivers who go to a bar more than once a month and are over the age of 25 to the all others.  Is there a difference?


5. Use the same process to compare the acceptance rate between drivers who go to bars more than once a month and had passengers that were not a kid and had occupations other than farming, fishing, or forestry.


6. Compare the acceptance rates between those drivers who:

- go to bars more than once a month, had passengers that were not a kid, and were not widowed *OR*
- go to bars more than once a month and are under the age of 30 *OR*
- go to cheap restaurants more than 4 times a month and income is less than 50K.



7.  Based on these observations, what do you hypothesize about drivers who accepted the bar coupons?

### Independent Investigation

Using the bar coupon example as motivation, you are to explore one of the other coupon groups and try to determine the characteristics of passengers who accept the coupons.  